# **LAB MID**

* Course: **Introduction to AI**
* Section: **BAI-A**
*    Group Members:
  1.    Malaika Junaid  **FA22-BAI-020**
  2.   Syeda Aleeza Tahir  **FA22-BAI-038**




##Game Problem:  **Tic-Tac-Toe**
###Description:
* Grid Size: The game is played on a 4x4 grid.
* Players: One human player and one computer player.
* Symbols: Players take turns marking symbols (●s or ■s) in empty cells.
* Target: The target remains to form a straight line with four of your symbols horizontally, vertically, or diagonally.

###Rules:
* Setup: The game is played on a 4x4 grid (16 cells) where two players take turns marking the cells. One player uses '●', and the other uses '■'.
* Gameplay: Players take turns placing their symbol in an empty cell until one player achieves four of their symbols in a row.
* Turns: Players alternate placing their symbol on the board. Once a cell is marked, it cannot be changed.
* Winning: The game can be won by getting four of your symbols in a row. This can be horizontally, vertically, or diagonally.
* Draw: If all cells are filled before either player achieves four in a row, the game is a draw.
* Goal: Don't lose - either Win or end up on draw.

###Objectives:
  - The primary objective is to form a sequence of four symbols in a row, either horizontally, vertically, or diagonally, before the opponent does.
  - Players aim to strategically place their symbols to either create winning sequences or block their opponent's attempts at achieving a winning sequence.
---

Tic Tac Toe - *without applying specific Strategies*

In [ ]:
import random

def print_board(board):
    print("   A   B   C   D")
    for i in range(4):
        print(str(i + 1) + "  " + " | ".join(board[i]))

def check_winner(board, player):
    # Check rows, columns, and diagonals for a win
    for i in range(4):
        # Check rows and columns
        if all(board[i][j] == player for j in range(4)) or all(board[j][i] == player for j in range(4)):
            return True
    # Check diagonals
    if all(board[i][i] == player for i in range(4)) or all(board[i][3 - i] == player for i in range(4)):
        return True
    return False

def available_moves(board):
    return [(i, j) for i in range(4) for j in range(4) if board[i][j] == ' ']

def player_turn(board, player):
    while True:
        move = input("Enter your move (e.g. A1, B2): ").upper()
        if len(move) != 2 or move[0] not in 'ABCD' or move[1] not in '1234':
            print("Invalid input. Please enter a valid move.")
            continue
        row = int(move[1]) - 1
        col = ord(move[0]) - ord('A')
        if board[row][col] == ' ':
            board[row][col] = player
            break
        else:
            print("That position is already taken. Try again.")

def computer_turn(board, player):
    print("Computer's turn:")
    available = available_moves(board)
    if available:
        # Try to win
        for move in available:
            board[move[0]][move[1]] = player
            if check_winner(board, player):
                return
            board[move[0]][move[1]] = ' '

        # Try to block player's win
        opponent = '●' if player == '■' else '■'
        for move in available:
            board[move[0]][move[1]] = opponent
            if check_winner(board, opponent):
                board[move[0]][move[1]] = player
                return
            board[move[0]][move[1]] = ' '

        # Choose a random move if no immediate win/block
        move = random.choice(available)
        board[move[0]][move[1]] = player

# Initialize the board
board = [[' ' for _ in range(4)] for _ in range(4)]

# Welcome message
print("Welcome to 4x4 Tic Tac Toe!\n")

# Ask for player's choice of symbol
player_symbol_c = int(input("Choose your symbol (1 for ■ or 2 for ●): "))
while player_symbol_c not in [1, 2]:
    player_symbol_c = int(input("Invalid input. Please choose either 1 or 2: "))
if player_symbol_c == 1:
    player_symbol = '■'
else:
    player_symbol = '●'

# Determine who goes first
player_turns = input("Do you want to go first? (yes or no): ").lower().startswith('y')

# Game loop
turns = 0
while turns < 16:
    print_board(board)
    if player_turns:
        player_turn(board, player_symbol)
        if check_winner(board, player_symbol):
            print_board(board)
            print("Congratulations! You win!")
            break
    else:
        computer_turn(board, '■' if player_symbol == '●' else '●')
        if check_winner(board, '■' if player_symbol == '●' else '●'):
            print_board(board)
            print("Computer wins! Better luck next time.")
            break
    turns += 1
    if turns == 16:
        print_board(board)
        print("It's a draw!")
    player_turns = not player_turns



---


##Tic Tac Toe - *using* ***Monte Carlo Tree Search***

MCTS (Monte Carlo Tree Search) dynamically explores the game space by conducting a four-phase process. This method intelligently samples potential moves, focusing on those demonstrating higher success rates, and expands unvisited nodes for further analysis. By simulating random games and updating node statistics, MCTS ensures a balanced consideration of possible outcomes, effectively guiding decision-making in a resource-efficient manner.

The components of MCTS implementation are stated below.
- Selection Phase:
  - The mcts_selection function selects the most promising child node from the current node based on the Upper Confidence Bounds for Trees (UCT) algorithm. It considers the exploration and exploitation of nodes. The exploration term ensures that all possible moves get sampled, while the exploitation term aids in favoring nodes that historically showed higher success rates.

- Expansion Phase:
  - The mcts_expansion function selects a random unvisited child node for expansion. It creates a new node representing a possible future game state given a random unexplored move.

- Simulation Phase:
  - The mcts_simulation function performs a random rollout or simulation of a game from the new node created in the expansion phase to determine a potential outcome. The game is simulated to completion with random moves to estimate the likelihood of success from a given node.

- Backpropagation Phase:
  - The mcts_backpropagation function backpropagates the result obtained from the simulation phase. It updates the statistics of the nodes in the tree, incrementing the visit count and win count, contributing to the exploration/exploitation balance.

The computer_turn function initially creates a tree, explores and expands nodes using selection and expansion, simulates games to evaluate potential outcomes, and then backpropagates the results to update the nodes.



###**AI Computer vs Human**

In [ ]:
import random
import math

def print_board(board):
    print("   A   B   C   D")
    for i in range(4):
        print(str(i + 1) + "  " + " | ".join(board[i]))

def check_winner(board, player):
    # Check rows, columns, and diagonals for a win
    for i in range(4):
        # Check rows and columns
        if all(board[i][j] == player for j in range(4)) or all(board[j][i] == player for j in range(4)):
            return True
    # Check diagonals
    if all(board[i][i] == player for i in range(4)) or all(board[i][3 - i] == player for i in range(4)):
        return True
    return False

def available_moves(board):
    return [(i, j) for i in range(4) for j in range(4) if board[i][j] == ' ']

def player_turn(board, player):
    while True:
        move = input("Enter your move (e.g. A1, B2): ").upper()
        if len(move) != 2 or move[0] not in 'ABCD' or move[1] not in '1234':
            print("Invalid input. Please enter a valid move.")
            continue
        row = int(move[1]) - 1
        col = ord(move[0]) - ord('A')
        if board[row][col] == ' ':
            board[row][col] = player
            break
        else:
            print("That position is already taken. Try again.")

class Node:
    def __init__(self, board, move=None, parent=None):
        self.board = board  # Store the current board state
        self.move = move  # Store the move that led to this state
        self.wins = 0  # Number of wins from this node
        self.visits = 0  # Number of visits to this node
        self.children = []  # List of child nodes
        self.parent = parent

def mcts_selection(node):
    C = 1.4  # Constant for exploration factor
    best_child = None
    best_score = -float('inf')

    for child in node.children:
        if child.visits == 0:
            score = float('inf')  # Assign a very high score for unvisited nodes
        else:
            exploitation_term = child.wins / child.visits
            if node.visits == 0 or child.visits == 0:
                exploration_term = C  # Set a default exploration value
            else:
                exploration_term = C * (math.sqrt(math.log(node.visits) / child.visits))
            score = exploitation_term + exploration_term

        if score > best_score:
            best_child = child
            best_score = score

    return best_child


def mcts_expansion(node):
    available_moves = [(i, j) for i in range(4) for j in range(4) if node.board[i][j] == ' ']
    if available_moves:
        move = random.choice(available_moves)
        new_board = [row[:] for row in node.board]  # Copy the board
        new_board[move[0]][move[1]] = '●'  # Assume AI always plays '●'
        new_node = Node(new_board, move, node)  # Include the parent node
        node.children.append(new_node)
        return new_node
    return None


def mcts_simulation(node):
    # Simulate random games (rollout) to determine potential outcomes
    simulated_board = [row[:] for row in node.board]  # Copy the board
    current_player = '■'  # Assuming human player always plays '■'
    winner = None

    while True:
        available_moves = [(i, j) for i in range(4) for j in range(4) if simulated_board[i][j] == ' ']
        if not available_moves or check_winner(simulated_board, '●') or check_winner(simulated_board, '■'):
            winner = check_winner(simulated_board, '●') or check_winner(simulated_board, '■')
            break
        move = random.choice(available_moves)
        simulated_board[move[0]][move[1]] = current_player
        current_player = '■' if current_player == '●' else '●'  # Switch players

    return 1 if winner == '●' else 0  # Assuming the goal is to maximize AI's wins


def mcts_backpropagation(node, result):
    # Update node statistics (wins and visits) based on the simulation result
    while node is not None:
        node.visits += 1
        node.wins += result
        node = node.parent


def computer_turn(board, player):
    root = Node(board)
    max_simulations = 1000

    for _ in range(max_simulations):
        node = root
        while node.children:
            node = mcts_selection(node)

        new_node = mcts_expansion(node)
        if new_node:
            result = mcts_simulation(new_node)
            mcts_backpropagation(new_node, result)

    # Check if AI can win or block the user from winning
    for i in range(4):
        for j in range(4):
            if board[i][j] == ' ':
                board[i][j] = '●'  # Assume AI's move

                # Check if AI can complete a line of 4
                if check_winner(board, '●'):
                    board[i][j] = player
                    return

                # Check if AI can block the player from completing a line of 4
                board[i][j] = '■' if player == '●' else '●'
                if check_winner(board, '■' if player == '●' else '●'):
                    board[i][j] = player
                    return

                board[i][j] = ' '  # Reset the position if no advantage
    # If no immediate win or block is possible, perform MCTS move selection
    best_child = max(root.children, key=lambda x: x.wins / x.visits)
    if best_child:
        row, col = best_child.move
        board[row][col] = player


# Initialize the board
board = [[' ' for _ in range(4)] for _ in range(4)]

# Welcome message
print("Welcome to 4x4 Tic Tac Toe!\n")

# Ask for player's choice of symbol
player_symbol_c = int(input("Choose your symbol (1 for ■ or 2 for ●): "))
while player_symbol_c not in [1, 2]:
    player_symbol_c = int(input("Invalid input. Please choose either 1 or 2: "))
if player_symbol_c == 1:
    player_symbol = '■'
else:
    player_symbol = '●'

# Determine who goes first
player_turns = input("Do you want to go first? (yes or no): ").lower().startswith('y')

# Game loop
turns = 0
while turns < 16:
    print_board(board)
    if player_turns:
        player_turn(board, player_symbol)
        if check_winner(board, player_symbol):
            print_board(board)
            print("Congratulations! You win!")
            break
    else:
        computer_turn(board, '■' if player_symbol == '●' else '●')
        if check_winner(board, '■' if player_symbol == '●' else '●'):
            print_board(board)
            print("Computer wins! Better luck next time.")
            break
    turns += 1
    if turns == 16:
        print_board(board)
        print("It's a draw!")
    player_turns = not player_turns

###**AI Computer 1 vs AI Computer 2**

In [ ]:
import random
import math

class Node:
    def __init__(self, board, move=None, parent=None):
        self.board = board
        self.move = move
        self.wins = 0
        self.visits = 0
        self.children = []
        self.parent = parent

def print_board(board):
    print("   A   B   C   D")
    for i in range(4):
        print(str(i + 1) + "  " + " | ".join(board[i]))

def check_winner(board, player):
    for i in range(4):
        if all(board[i][j] == player for j in range(4)) or all(board[j][i] == player for j in range(4)):
            return True
    if all(board[i][i] == player for i in range(4)) or all(board[i][3 - i] == player for i in range(4)):
        return True
    return False

def available_moves(board):
    return [(i, j) for i in range(4) for j in range(4) if board[i][j] == ' ']

def mcts_selection(node):
    C = 1.4
    best_child = None
    best_score = -float('inf')

    for child in node.children:
        if child.visits == 0:
            score = float('inf')
        else:
            exploitation_term = child.wins / child.visits
            if node.visits == 0 or child.visits == 0:
                exploration_term = C
            else:
                exploration_term = C * (math.sqrt(math.log(node.visits) / child.visits))
            score = exploitation_term + exploration_term

        if score > best_score:
            best_child = child
            best_score = score

    return best_child

def mcts_expansion(node):
    available_moves = [(i, j) for i in range(4) for j in range(4) if node.board[i][j] == ' ']
    if available_moves:
        move = random.choice(available_moves)
        new_board = [row[:] for row in node.board]
        new_board[move[0]][move[1]] = '●'
        new_node = Node(new_board, move, node)
        node.children.append(new_node)
        return new_node
    return None

def mcts_simulation(node):
    simulated_board = [row[:] for row in node.board]
    current_player = '■'
    winner = None

    while True:
        available_moves = [(i, j) for i in range(4) for j in range(4) if simulated_board[i][j] == ' ']
        if not available_moves or check_winner(simulated_board, '●') or check_winner(simulated_board, '■'):
            winner = check_winner(simulated_board, '●') or check_winner(simulated_board, '■')
            break
        move = random.choice(available_moves)
        simulated_board[move[0]][move[1]] = current_player
        current_player = '■' if current_player == '●' else '●'

    return 1 if winner == '●' else 0

def mcts_backpropagation(node, result):
    while node is not None:
        node.visits += 1
        node.wins += result
        node = node.parent

def defensive_computer_turn(board, player):
    root = Node(board)
    max_simulations = 1000

    for _ in range(max_simulations):
        node = root
        while node.children:
            node = mcts_selection(node)

        new_node = mcts_expansion(node)
        if new_node:
            result = mcts_simulation(new_node)
            mcts_backpropagation(new_node, result)

    best_child = max(root.children, key=lambda x: x.wins / x.visits)
    if best_child:
        row, col = best_child.move
        board[row][col] = player

def offensive_computer_turn(board, player):
    root = Node(board)
    max_simulations = 1000

    for _ in range(max_simulations):
        node = root
        while node.children:
            node = mcts_selection(node)

        new_node = mcts_expansion(node)
        if new_node:
            result = mcts_simulation(new_node)
            mcts_backpropagation(new_node, result)

    best_child = max(root.children, key=lambda x: x.wins / x.visits)
    if best_child:
        row, col = best_child.move
        board[row][col] = player

def ask_user():
    user_input = input("Enter 1 for AI1 (Defensive) to start or 2 for AI2 (Offensive) to start: ")
    if user_input in ['1', '2']:
        return user_input
    return ask_user()

# Initialize the board
board = [[' ' for _ in range(4)] for _ in range(4)]

first_turn = ask_user()
turns = 0
while turns < 16:
    print_board(board)
    if first_turn == '1':
        if turns % 2 == 0:
            print("\nAI1 (Defensive) turn:\n")
            defensive_computer_turn(board, '■')
            if check_winner(board, '■'):
                print_board(board)
                print("\nAI1 (Defensive) wins!")
                break
        else:
            print("\nAI2 (Offensive) turn:\n")
            offensive_computer_turn(board, '●')
            if check_winner(board, '●'):
                print_board(board)
                print("AI2 (Offensive) wins!")
                break
    else:
        if turns % 2 == 0:
            print("\nAI2 (Offensive) turn:\n")
            offensive_computer_turn(board, '●')
            if check_winner(board, '●'):
                print_board(board)
                print("AI2 (Offensive) wins!")
                break
        else:
            print("\nAI1 (Defensive) turn:\n")
            defensive_computer_turn(board, '■')
            if check_winner(board, '■'):
                print_board(board)
                print("\nAI1 (Defensive) wins!")
                break
    turns += 1
if turns == 16:
    print_board(board)
    print("It's a draw!")



---


##Tic Tac Toe - *using* ***Alpha-Beta Prunning***

Alpha-Beta Pruning significantly minimizes the number of explored nodes by discarding unpromising moves and their subsequent branches. This enhancement improves search efficiency without compromising the accuracy of the decision-making process.



The Alpha-Beta Pruning algorithm optimizes the search process by:

- Minimizing explored nodes:
    -  It prunes unnecessary branches in the search tree where the best moves have been determined. This is achieved by maintaining alpha and beta values, representing the best possible outcomes for the maximizing and minimizing players. These values help discard irrelevant moves, reducing the number of nodes explored.

- Pruning irrelevant subtrees:
  - If a move does not influence the final decision (not changing alpha or beta values), the search in that direction is cut off. This significantly reduces the search space by ignoring branches that won't affect the final decision, boosting efficiency.

- Depth limitation:
  - By limiting the search depth to 3 in this implementation, the algorithm focuses on a subset of possible moves, which reduces the tree size. This strategy helps the algorithm make informed decisions within resource constraints.


###**AI Computer vs Human**


In [ ]:
import random
import math

def print_board(board):
    print("   A   B   C   D")
    for i in range(4):
        print(str(i + 1) + "  " + " | ".join(board[i]))

def check_winner(board, player):
    # Check rows, columns, and diagonals for a win
    for i in range(4):
        # Check rows and columns
        if all(board[i][j] == player for j in range(4)) or all(board[j][i] == player for j in range(4)):
            return True
    # Check diagonals
    if all(board[i][i] == player for i in range(4)) or all(board[i][3 - i] == player for i in range(4)):
        return True
    return False

def available_moves(board):
    return [(i, j) for i in range(4) for j in range(4) if board[i][j] == ' ']

def player_turn(board, player):
    while True:
        move = input("Enter your move (e.g. A1, B2): ").upper()
        if len(move) != 2 or move[0] not in 'ABCD' or move[1] not in '1234':
            print("Invalid input. Please enter a valid move.")
            continue
        row = int(move[1]) - 1
        col = ord(move[0]) - ord('A')
        if board[row][col] == ' ':
            board[row][col] = player
            break
        else:
            print("That position is already taken. Try again.")

def evaluate(board, player):
    # This function returns a score indicating how good the current board state is for the given player.

    # Define a mapping for the players
    player_map = {'●': -1, '■': 1, ' ': 0}

    # Score the board
    score = 0

    # Evaluate rows, columns, and diagonals
    for i in range(4):
        # Rows and columns
        row_values = [player_map[board[i][j]] for j in range(4)]
        col_values = [player_map[board[j][i]] for j in range(4)]
        score += row_values.count(player_map[player]) + col_values.count(player_map[player])

    # Diagonals
    diag1 = [player_map[board[i][i]] for i in range(4)]
    diag2 = [player_map[board[i][3 - i]] for i in range(4)]
    score += diag1.count(player_map[player]) + diag2.count(player_map[player])

    return score

def alpha_beta(board, depth, alpha, beta, maximizing_player, player):
    if depth == 0 or check_winner(board, '●') or check_winner(board, '■'):
        # Evaluate the board state and return the score
        return evaluate(board, player)

    if maximizing_player:
        max_eval = float('-inf')
        for move in available_moves(board):
            board[move[0]][move[1]] = '●'
            eval = alpha_beta(board, depth - 1, alpha, beta, False, player)
            board[move[0]][move[1]] = ' '  # Undo the move
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break  # Beta cut-off
        return max_eval
    else:
        min_eval = float('inf')
        for move in available_moves(board):
            board[move[0]][move[1]] = '■'
            eval = alpha_beta(board, depth - 1, alpha, beta, True, player)
            board[move[0]][move[1]] = ' '  # Undo the move
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break  # Alpha cut-off
        return min_eval

def computer_turn(board, player):
    print("Computer's turn:")
    best_val = -float('inf')
    best_move = None
    block_move = None  # Record to block the opponent's winning move

    for move in available_moves(board):
        board[move[0]][move[1]] = player
        # Check if this move leads to the computer's win
        if check_winner(board, player):
            board[move[0]][move[1]] = ' '  # Undo the move
            board[move[0]][move[1]] = player
            return

        # Check if this move can block the opponent's winning move
        opponent = '●' if player == '■' else '■'
        board[move[0]][move[1]] = opponent
        if check_winner(board, opponent):
            block_move = move
        board[move[0]][move[1]] = ' '  # Undo the move

        board[move[0]][move[1]] = player
        val = alpha_beta(board, 3, -float('inf'), float('inf'), False, player)
        board[move[0]][move[1]] = ' '

        if val > best_val:
            best_val = val
            best_move = move

    # If there's a move that blocks the opponent, take that move, else proceed with alpha-beta evaluation
    if block_move:
        board[block_move[0]][block_move[1]] = player
    elif best_move:
        board[best_move[0]][best_move[1]] = player


# Initialize the board
board = [[' ' for _ in range(4)] for _ in range(4)]

# Welcome message
print("Welcome to 4x4 Tic Tac Toe!\n")

# Ask for player's choice of symbol
player_symbol_c = int(input("Choose your symbol (1 for ■ or 2 for ●): "))
while player_symbol_c not in [1, 2]:
    player_symbol_c = int(input("Invalid input. Please choose either 1 or 2: "))
if player_symbol_c == 1:
    player_symbol = '■'
else:
    player_symbol = '●'

# Determine who goes first
player_turns = input("Do you want to go first? (yes or no): ").lower().startswith('y')

# Game loop
turns = 0
while turns < 16:
    print_board(board)
    if player_turns:
        player_turn(board, player_symbol)
        if check_winner(board, player_symbol):
            print_board(board)
            print("Congratulations! You win!")
            break
    else:
        computer_turn(board, '■' if player_symbol == '●' else '●')
        if check_winner(board, '■' if player_symbol == '●' else '●'):
            print_board(board)
            print("Computer wins! Better luck next time.")
            break
    turns += 1
    if turns == 16:
        print_board(board)
        print("It's a draw!")
    player_turns = not player_turns



###**AI Computer 1 vs AI Computer 2**

In [ ]:
import random
import math

def print_board(board):
    print("   A   B   C   D")
    for i in range(4):
        print(str(i + 1) + "  " + " | ".join(board[i]))

def check_winner(board, player):
    for i in range(4):
        if all(board[i][j] == player for j in range(4)) or all(board[j][i] == player for j in range(4)):
            return True
    if all(board[i][i] == player for i in range(4)) or all(board[i][3 - i] == player for i in range(4)):
        return True
    return False

def available_moves(board):
    return [(i, j) for i in range(4) for j in range(4) if board[i][j] == ' ']

def evaluate_defensive(board, player):
    player_map = {'●': -1, '■': 1, ' ': 0}
    score = 0
    for i in range(4):
        row_values = [player_map[board[i][j]] for j in range(4)]
        col_values = [player_map[board[j][i]] for j in range(4)]
        score += row_values.count(player_map[player]) + col_values.count(player_map[player])
    diag1 = [player_map[board[i][i]] for i in range(4)]
    diag2 = [player_map[board[i][3 - i]] for i in range(4)]
    score += diag1.count(player_map[player]) + diag2.count(player_map[player])
    return score

def evaluate_offensive(board, player):
    player_map = {'●': -1, '■': 1, ' ': 0}
    score = 0
    for i in range(4):
        row_values = [player_map[board[i][j]] for j in range(4)]
        col_values = [player_map[board[j][i]] for j in range(4)]
        score += row_values.count(player_map[player]) + col_values.count(player_map[player])
    diag1 = [player_map[board[i][i]] for i in range(4)]
    diag2 = [player_map[board[i][3 - i]] for i in range(4)]
    score += diag1.count(player_map[player]) + diag2.count(player_map[player])
    return score

def alpha_beta_defensive(board, depth, alpha, beta, maximizing_player, player):
    if depth == 0 or check_winner(board, '●') or check_winner(board, '■'):
        return evaluate_defensive(board, player)

    if maximizing_player:
        max_eval = float('-inf')
        for move in available_moves(board):
            board[move[0]][move[1]] = '●'
            eval = alpha_beta_defensive(board, depth - 1, alpha, beta, False, player)
            board[move[0]][move[1]] = ' '
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in available_moves(board):
            board[move[0]][move[1]] = '■'
            eval = alpha_beta_defensive(board, depth - 1, alpha, beta, True, player)
            board[move[0]][move[1]] = ' '
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def alpha_beta_offensive(board, depth, alpha, beta, maximizing_player, player):
    if depth == 0 or check_winner(board, '●') or check_winner(board, '■'):
        return evaluate_offensive(board, player)

    if maximizing_player:
        max_eval = float('-inf')
        for move in available_moves(board):
            board[move[0]][move[1]] = '■'
            eval = alpha_beta_offensive(board, depth - 1, alpha, beta, False, player)
            board[move[0]][move[1]] = ' '
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = float('inf')
        for move in available_moves(board):
            board[move[0]][move[1]] = '●'
            eval = alpha_beta_offensive(board, depth - 1, alpha, beta, True, player)
            board[move[0]][move[1]] = ' '
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def defensive_computer_turn(board, player):
    best_val = -float('inf')
    best_move = None
    block_move = None
    for move in available_moves(board):
        board[move[0]][move[1]] = player
        if check_winner(board, player):
            board[move[0]][move[1]] = ' '
            return
        opponent = '●' if player == '■' else '■'
        board[move[0]][move[1]] = opponent
        if check_winner(board, opponent):
            block_move = move
        board[move[0]][move[1]] = ' '
        board[move[0]][move[1]] = player
        val = alpha_beta_defensive(board, 3, -float('inf'), float('inf'), False, player)
        board[move[0]][move[1]] = ' '
        if val > best_val:
            best_val = val
            best_move = move
    if block_move:
        board[block_move[0]][block_move[1]] = player
    elif best_move:
        board[best_move[0]][best_move[1]] = player

def offensive_computer_turn(board, player):
    best_val = -float('inf')
    best_move = None
    for move in available_moves(board):
        board[move[0]][move[1]] = player
        val = alpha_beta_offensive(board, 4, -float('inf'), float('inf'), False, player)
        board[move[0]][move[1]] = ' '
        if val > best_val:
            best_val = val
            best_move = move
    if best_move:
        board[best_move[0]][best_move[1]] = player

# Initialize the board
board = [[' ' for _ in range(4)] for _ in range(4)]

# Welcome message
print("Welcome to 4x4 Tic Tac Toe!\n")

# Ask the user to choose their symbol
offensive_symbol = input("Choose AI 1 symbol (1 for '●' or 2 for '■'): ")
while offensive_symbol not in ['1', '2']:
    offensive_symbol = input("Invalid input. Please choose either 1 or 2: ")
if offensive_symbol == '1':
    offensive_player = '●'
    defensive_player = '■'
else:
    offensive_player = '■'
    defensive_player = '●'

# Game loop between two AI players
turns = 0
while turns < 16:
    print_board(board)
    if turns % 2 == 0:
        print("\nAI1 (O) turn:\n")
        offensive_computer_turn(board, offensive_player)
        if check_winner(board, offensive_player):
            print_board(board)
            print("\nAI1 (O) wins!")
            break
    else:
        print("\nAI2 (D) turn:\n")
        defensive_computer_turn(board, defensive_player)
        if check_winner(board, defensive_player):
            print_board(board)
            print("AI2 (D) wins!")
            break
    turns += 1
if turns == 16:
    print_board(board)
    print("It's a draw!")



---


##**Comparative Analysis**


*   **Efficiency**: *(Number of Nodes Explored)*
  - MCTS:
      - MCTS explores a varied number of nodes and uses random simulations to decide the most promising ones.
      - The number of nodes visited increases with the number of simulations.
      - Can explore a large number of nodes, especially in complex games, requiring more computational resources.
  - Alpha-Beta Pruning:
      - Reduces the number of explored nodes by eliminating unnecessary branches, leading to more efficient exploration.
      - Effectiveness increases with a higher depth search and a lower branching factor.
      - Even though it prunes nodes, it can still explore a substantial number, particularly in games with a vast search space.


*   **Accuracy**: *(Quality of Moves Chosen)*
  - MCTS:
      - Focuses on exploring a wide range of moves, providing high-quality moves with a higher number of simulations.
      - Works well when there's a large number of possible moves.
      - Accuracy depends heavily on the number of simulations, sometimes making suboptimal moves with an insufficient number of simulations.
  - Alpha-Beta Pruning:
      - Aims to choose the best moves by exploring deeper and considering a limited number of nodes due to pruning.
      - Quality of moves depends on the effectiveness of the heuristic evaluation function.
      - May overlook certain good moves due to cutoffs or ineffective evaluation functions.


##**Results and Conclusion**

*   **Insights**:
  - MCTS Insights:
      - Effective in exploring numerous possible moves due to its randomized exploration.
      - Relatively computationally expensive, especially in scenarios with an extensive number of possible moves, requiring more simulations for accurate decisions.

  - Alpha-Beta Pruning Insights:
      - Efficient in reducing the number of nodes explored.
      - Can sometimes overlook potential good moves based on the heuristic evaluation, especially in complex game states.
    
*   **Verdict**:
  - MCTS:
      - Effective in scenarios with numerous possible moves due to its randomized exploration. However, it requires a significant number of simulations for reasonable decisions.
  - Alpha-Beta Pruning:
      - Performs relatively well, especially with a limited number of moves, exploring a smaller subset of nodes.

*   **Conclusion**:
      -  In a nutshell, while both alpha-beta pruning and MCTS could be technically applied to Tic Tac Toe, alpha-beta pruning is more suitable and efficient given the game's simplicity and deterministic nature. MCTS introduces unnecessary computational overhead for a game with a small and manageable decision space like Tic Tac Toe.

---

